In [2]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

In [11]:
flights = pd.read_csv("flights_csv.csv").drop(columns="Unnamed: 0")
flights.head(20)

,airport_from,airport_to,city_from,city_to,distance,price,nights,seats,airlines,departure
0,BCN,PMI,Barcelona,"Palma, Majorca",202.18,31,9,3,VY,2024-05-07
1,BCN,PMI,Barcelona,"Palma, Majorca",202.18,31,10,1,VY,2024-05-06
2,BCN,PMI,Barcelona,"Palma, Majorca",202.18,31,5,3,VY,2024-05-11
3,BCN,PMI,Barcelona,"Palma, Majorca",202.18,66,3,9,UX,2024-05-06
4,BCN,PMI,Barcelona,"Palma, Majorca",202.18,66,3,4,UX,2024-05-11
5,BCN,PMI,Barcelona,"Palma, Majorca",202.18,66,9,2,UX,2024-05-06
6,BCN,CDG,Barcelona,Paris,859.37,187,9,5,AF,2024-05-05
7,BCN,CDG,Barcelona,Paris,859.37,187,5,7,AF,2024-05-05
8,BCN,CDG,Barcelona,Paris,859.37,187,6,6,AF,2024-05-05
9,BCN,LHR,Barcelona,London,1149.76,207,4,9,IB,2024-05-09


In [9]:
# The below is not a great idea since it disconnets the row values from each other
cheapest_per_combo = flights.groupby(["airport_from", "airport_to"]).agg("min")
cheapest_per_combo.head(1)

,,Unnamed: 0,city_from,city_to,distance,price,nights,seats,airlines,departure
airport_from,airport_to,,,,,,,,,
BCN,CDG,6,Barcelona,Paris,859.37,187,5,5,AF,2024-05-05


In [18]:
cheapest_combos = (
    flights.sort_values(by="price", ascending=True)
    .groupby(["city_from", "city_to"])
    .agg("first")
)
cheapest_combos

airport_from airport_to  distance  price  nights  \
city_from  city_to                                                           
Barcelona  London                  BCN        LHR   1149.76    207       4   
           Munich                  BCN        MUC   1095.88    291       5   
           Palma, Majorca          BCN        PMI    202.18     31       9   
           Paris                   BCN        CDG    859.37    187       9   
           Vienna                  BCN        VIE   1371.79    227       5   
Gothenburg Copenhagen              GOT        CPH    228.50    218       7   
           London                  GOT        LGW   1082.70     50       3   
           Munich                  GOT        MUC   1036.78    223       8   
           Palma, Majorca          GOT        PMI   2130.07    674       4   
           Paris                   GOT        CDG   1157.83    314       4   
           Vienna                  GOT        VIE   1101.32    306       9   
Helsinki   London                  HEL        LGW   1860.32    135       3   
           Munich                  HEL        MUC   1576.30    325       8   
           Málaga                  HEL        AGP   3356.39    262       5   
           Palma, Majorca          HEL        PMI   2777.47    335       5   
           Paris                   HEL        CDG   1898.05    158       7   
           Vienna                  HEL        VIE   1461.67    407       4   

                           seats airlines   departure  
city_from  city_to                                     
Barcelona  London              9       IB  2024-05-09  
           Munich              9       LH  2024-05-10  
           Palma, Majorca      3       VY  2024-05-07  
           Paris               5       AF  2024-05-05  
           Vienna              9       OS  2024-05-08  
Gothenburg Copenhagen          9       SK  2024-05-11  
           London              9  W6 & D8  2024-05-12  
           Munich              5       LH  2024-05-08  
           Palma, Majorca      9       SK  2024-05-07  
           Paris               4       AF  2024-05-05  
           Vienna              9       OS  2024-05-10  
Helsinki   London              6       D8  2024-05-10  
           Munich              9       LH  2024-05-10  
           Málaga              2       D8  2024-05-12  
           Palma, Majorca      4       D8  2024-05-09  
           Paris               1  D8 & AY  2024-05-06  
           Vienna              7       AY  2024-05-10

In [24]:
group_destinations = cheapest_combos.groupby(["city_to"]).agg({"price": "sum"}).sort_values("price").reset_index()
group_destinations

,city_to,price
0,Copenhagen,218
1,Málaga,262
2,London,392
3,Paris,659
4,Munich,839
5,Vienna,940
6,"Palma, Majorca",1040


In [28]:
cheapest_group_destination = group_destinations.iloc[0]
print(f'The cheapetst destination for you and your friends to meet at is {cheapest_group_destination["city_to"]}, '
      f'at €{cheapest_group_destination["price"]} in total or €{round(cheapest_group_destination["price"]/3),2} per person.')

SyntaxError: f-string: unmatched '[' (1175831145.py, line 2)

In [6]:
load_dotenv()
kiwi_user = os.environ.get("KIWI_USER")
kiwi_key = os.environ.get("KIWI_KEY")
base_url = "https://api.tequila.kiwi.com"
headers = {"Content-Type": "application/json", "apikey": kiwi_key}

In [97]:
params = {
    "term": "",
    "limit": 500,
    "sort": "name",
    "active_only": True,
    "source_popularity": "bookings",
}
origins = ["gothenburg_se", "barcelona_es", "paris_fr", "berlin_de"]
for i, city in enumerate(origins):
    params["term"] = city
    res = requests.get(
        f"{base_url}/locations/topdestinations", params=params, headers=headers
    )
    if res.status_code == 200:
        data = res.json()
        print(f"{data['results_retrieved']} results retrieved for {city}")
        retrieved_locations = {
            location["id"]
            for location in data["locations"]
        }
        retrieved_locations.add(city)
        if i == 0:
            all_results = retrieved_locations
        else:
            all_results = all_results.intersection(retrieved_locations)
    else:
        print(f"Something went wrong for {city}! Status: {res.status_code}")

104 results retrieved for gothenburg_se
250 results retrieved for barcelona_es
250 results retrieved for paris_fr
250 results retrieved for berlin_de


In [98]:
print("Total number of common destinations:", len(all_results))
print(all_results)

Total number of common destinations: 67
{'las-palmas_es', 'brussels_be', 'naples_it', 'amman_jo', 'tirana_al', 'ibiza_es', 'larnaca_cy', 'antalya_tr', 'cairo_eg', 'riyadh_sa', 'catania_it', 'riga_lv', 'santorini_gr', 'amsterdam_nl', 'copenhagen_dk', 'rome_it', 'bangkok_th', 'warsaw_pl', 'olbia_it', 'bergen_no', 'pristina_xk', 'rhodes_gr', 'alicante_es', 'dublin_ie', 'palma_es', 'lisbon_pt', 'sharm-el-sheikh_eg', 'tangier_ma', 'prague_cz', 'dusseldorf_de', 'helsinki_fi', 'malta_mt', 'malaga_es', 'yerevan_am', 'dubrovnik_hr', 'funchal_pt', 'gdansk_pl', 'hamburg_de', 'madrid_es', 'geneva_ch', 'bucharest_ro', 'miami_fl_us', 'jeddah_sa', 'paris_fr', 'skopje_mk', 'istanbul_tr', 'nantes_fr', 'bilbao_es', 'krakow_pl', 'stockholm_se', 'berlin_de', 'tenerife_es', 'munich_de', 'athens_gr', 'london_gb', 'los-angeles_ca_us', 'vilnius_lt', 'belgrade_rs', 'bristol_gb', 'milan_it', 'thessaloniki_gr', 'strasbourg_fr', 'venice_it', 'frankfurt_de', 'tallinn_ee', 'suceava_ro', 'barcelona_es'}


In [96]:
endpoint = f"{base_url}/locations"
query = f"/query?term=London"
response = requests.get(url=endpoint + query, headers=headers)
response.raise_for_status()
city_data = response.json()
print(response.status_code)
print(city_data)

200
{'locations': [{'id': 'london_gb', 'active': True, 'name': 'London', 'slug': 'london-united-kingdom', 'slug_en': 'london-united-kingdom', 'code': 'LON', 'alternative_names': [], 'rank': 1, 'global_rank_dst': 0, 'dst_popularity_score': 10744937.0, 'timezone': 'Europe/London', 'population': 7556900, 'airports': 6, 'stations': 6, 'hotels': 8104, 'bus_stations': 26, 'subdivision': {'id': 'EN_GB', 'name': 'England', 'slug': 'england-united-kingdom', 'code': 'ENG'}, 'autonomous_territory': None, 'country': {'id': 'GB', 'name': 'United Kingdom', 'slug': 'united-kingdom', 'code': 'GB'}, 'region': {'id': 'northern-europe', 'name': 'Northern Europe', 'slug': 'northern-europe'}, 'continent': {'id': 'europe', 'name': 'Europe', 'slug': 'europe', 'code': 'EU'}, 'nearby_country': None, 'location': {'lat': 51.507351, 'lon': -0.127758}, 'tags': [{'tag': 'activities', 'month_to': -1, 'month_from': -1}, {'tag': 'city break', 'month_to': -1, 'month_from': -1}, {'tag': 'events', 'month_to': -1, 'month_

In [127]:
import datetime as dt

endpoint = f"{base_url}/v2/search?query="
max_price = 1000
today = dt.datetime.today()
months_from_today = today + dt.timedelta(days=30)
#from_date = self.today.strftime("%d/%m/%Y")
#to_date = self.months_from_today.strftime("%d/%m/%Y")
#stopovers = int(city_data["stopovers"])
params = {
    "fly_from": "paris_fr",
    "fly_to": "berlin_de",
    "date_from": (today + dt.timedelta(days=7)).strftime("%d/%m/%Y"),
    "date_to": (today + dt.timedelta(days=14)).strftime("%d/%m/%Y"),
    "price_to": max_price,
    "nights_in_dst_from": 3,
    "nights_in_dst_to": 10,
    "one_for_city": True,
    "one_per_date": True,
    "curr": "EUR",
    "max_stopovers": 0,
    "limit": 500
}    
response = requests.get(url=endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()["data"]

In [123]:
len(data)

44

In [128]:
print(response.json())

{'search_id': '38da0866-6a43-10a5-1b7a-f03d114df96c', 'currency': 'EUR', 'fx_rate': 1, 'data': [{'id': '0a7c00424d8c000095133bb0_0|00420a7c4d92000003a9a712_0', 'flyFrom': 'ORY', 'flyTo': 'BER', 'cityFrom': 'Paris', 'cityCodeFrom': 'PAR', 'cityTo': 'Berlin', 'cityCodeTo': 'BER', 'countryFrom': {'code': 'FR', 'name': 'France'}, 'countryTo': {'code': 'DE', 'name': 'Germany'}, 'local_departure': '2024-05-09T21:20:00.000Z', 'utc_departure': '2024-05-09T19:20:00.000Z', 'local_arrival': '2024-05-09T23:10:00.000Z', 'utc_arrival': '2024-05-09T21:10:00.000Z', 'nightsInDest': 6, 'quality': 190.3332, 'distance': 883.98, 'duration': {'departure': 6600, 'return': 6600, 'total': 13200}, 'price': 137, 'conversion': {'EUR': 137}, 'fare': {'adults': 137, 'children': 137, 'infants': 137}, 'fare_locks': {'EUR': [{'default': False, 'duration': 'P7D', 'itinerary_price_limit': 337, 'itinerary_price_remaining': 137, 'fare_lock_kind': 'fee', 'price': 34.25, 'rule_instance_id': 26750, 'version': 'v1'}, {'defaul

In [124]:
data

for d in data:
    print(d["cityTo"])

Milan
London
London
Milan
Dublin
Dublin
Prague
Seville
London
Madrid
Naples
Palma, Majorca
Berlin
Milan
Tangier
Dublin
Prague
Barcelona
Tunis
Istanbul
Venice
Venice
Seville
Warsaw
Seville
Marrakesh
Agadir
Heraklion
Cairo
Cairo
New York
Los Angeles
Los Angeles
Los Angeles
Los Angeles
San Francisco
Los Angeles
Larnaca
Ibiza
Ibiza
Dubai
Palermo
Palermo
Larnaca
